# Spikey Tutorial

A narrow introduction to Spikey with examples.

Table of Contents
---
1. Installation
2. Pieces of a Spiking Neural Network
3. Creating a Spiking Neural Network
4. Games
5. Training Loops
6. Tracking Signals
7. Moving Forward

```none
----------  -----------  ---------  -----
| Neuron |  | Synapse |  | Input |  | ...
----------  -----------  ---------  -----
       \         |         /
         \       |       /
--------   -------------
| Game |   |  Network  |
--------   -------------    ------------
   |____________/___________| Callback | --> Logger --> Reader
   |           /            ------------
-----------------
| Training Loop |
-----------------
        |
----------------------
| Aggregate Analysis |
----------------------
    ^       |
    L_______|
```

## Installation

First open a terminal in the root directory of your cloned Spikey repo.

You should see something like,
```bash
.../username/spikey>
```

If not change terminals working directory to the spikey repo via,
```bash
cd <path to your cloned spikey repo>

# eg,
cd /home/user/spikey
```

Install all necessary dependencies,
```bash
pip install -r requirements.txt
```

Then since Spikey is not yet on pypi, install it locally via
```bash
pip install -e .
```

Spikey only needs to be reinstalled if you re-clone the repo or move it.

In [ ]:
# After installation, restart this notebook and this import should work.
import spikey

## Pieces of a spiking neural network

Spiking neural networks are complex systems capable of learning in a similar way to the human brain.
They are composed of many unique parts: neurons, synapses, ... all of which are individually simple, 
yet can cooperate to achieve the ability to understand the environment they operate in.
This is an emergent process.

In this simulator we have individual objects for all spiking network parts.
There are multiple pre-built versions of each, but you may template any of them to create your own versions.

Luckily the Network object exists within Spikey to manage all of these parts for you, but we will kick of the tutorial by demoing them individually.
```
Neuron: Standard neuron behavior.
Synapse: Synaptic learning rule implementation.
Weight: Datastructure to generate and manage network topology, connects individual inputs and neurons together.
Input: Neuron without inputs that serve to encode sensory information from the environment.
Readout: Function to translate the network's reaction into an action within the environment.
Rewarder(RL networks only): Determine how much reward the network deserves for its response.
```
```none
----------  -----------  ---------  -----
| Neuron |  | Synapse |  | Input |  | ...
----------  -----------  ---------  -----
       \         |         /
         \       |       /
           -------------
           |  Network  |
           -------------
```

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from time import time
import os
from spikey import Key

In [ ]:
# Each piece of the network has many parameters that need to be set.
# All parameters and their descriptions are listed in the parts NECESSARY_KEYS.
input_type = spikey.input.StaticMap
input_type.NECESSARY_KEYS

In [ ]:
# Keys can be configured by a dictionary to be passed into the constructor
input_config = {
    'n_inputs': 2,
    'magnitude': 1,
    'firing_steps': -1,
    'input_pct_inhibitory': 0,
    'mapping': {0: [0, 0], 1: [0, 1], 2: [1, 0], 3: [1, 1]},
}
input = input_type(**input_config)
input.reset()

In [ ]:
# Tell the input about the environment state
input.update(1)

# Input fire according to state
input()

In [ ]:
neuron_type = spikey.neuron.Neuron
neuron_type.NECESSARY_KEYS

In [ ]:
# Each part manages the whole set of such pieces, eg the Neuron part may contain 100+ neurons.
neuron_config = {
    'magnitude': 1.5,
    'n_neurons': 4,
    'neuron_pct_inhibitory': 0.,
    'potential_decay': .25,
    'prob_rand_fire': 0.,
    'refractory_period': 1,
    'resting_mv': 0.,
    'firing_threshold': 2.,
    'spike_delay': 0,
}
neuron = neuron_type(**neuron_config)
neuron.reset()

neuron.potentials

In [ ]:
# Add potentials then see which fire
neuron += np.full(neuron_config['n_neurons'],  10)
fires = neuron >= neuron_config['firing_threshold']
fires

In [ ]:
# Every piece of this simulator can be inhereted and extended or modified according to the user's needs.
# Each object contains a base template (eg spikey.weight.template.Weight) that defines bare minimum expected functionality.
from spikey.weight.template import Weight

class CustomWeight(Weight):
    NECESSARY_KEYS = Weight.extend_keys([
        Key('matrix', 'np.ma.array[n_inputs+n_neurons, n_neurons] Connections between individual inputs and neurons.', type=np.ndarray)
    ])
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self._matrix = np.ma.copy(kwargs['matrix'])
        assert self._matrix.shape == (self._n_inputs+self._n_neurons, self._n_neurons), \
            "Incorrect weight shape!"
        self._clip_weights()

CustomWeight.NECESSARY_KEYS

In [ ]:
n_inputs = input_config['n_inputs']
n_neurons = neuron_config['n_neurons']
weight_config = {
    "n_inputs": n_inputs,
    "n_neurons": n_neurons,
    "max_weight": 1,
    'matrix': np.ones((n_inputs+n_neurons, n_neurons))
}
weight = CustomWeight(**weight_config)

In [ ]:
# A bare bones spiking network loop
N_STEP = 10

input.reset()
neuron.reset()

spike_log = np.empty((N_STEP, n_inputs + n_neurons))
for s in range(N_STEP):
    input.update(np.random.randint(4))

    spikes = np.append(input(), neuron >= neuron_config['firing_threshold'])
    neuron.update()

    neuron += np.matmul(weight.matrix.T, spikes)

    spike_log[s] = spikes

plt.imshow(spike_log, cmap='gray')
plt.show()

## Creating a Spiking Neural Network

Network is a neat interface for the many pieces that actually make up a spiking neural network.
Simply pass it a list of part types with their configurations and it's ready to train.

```none
----------  -----------  ---------  -----
| Neuron |  | Synapse |  | Input |  | ...
----------  -----------  ---------  -----
       \         |         /
         \       |       /
           -------------
           |  Network  |
           -------------
```

In [ ]:
# Similar to individual piece's NECESSARY_KEYS, the Network has NECESSARY_PARTS
network_type = spikey.network.RLNetwork
network_type.NECESSARY_PARTS

In [ ]:
parts = {
    'inputs': spikey.input.StaticMap,
    'neurons': spikey.neuron.Neuron,
    'weights': spikey.weight.Manual,
    'synapses': spikey.synapse.RLSTDPET,
    'readout': spikey.readout.NeuronRates,
    'modifiers': None,
    'rewarder': spikey.reward.MatchExpected,
}

network_type.list_keys(**parts)

In [ ]:
network_config = {
    "n_inputs": 2,
	"n_neurons": 4,
	"n_outputs": 4,
	"processing_time": 10,

	"magnitude": 1,
	"resting_mv": 0,
	"firing_threshold": 2,
	"spike_delay": 0,
	"refractory_period": 1,
	"potential_decay": .25,
	"prob_rand_fire": 0,
	"neuron_pct_inhibitory": 0,

	"learning_rate": .01,
	"stdp_window": 20,
	"trace_decay": .25,
	"max_weight": 1,
	"matrix": np.random.uniform(size=(6, 4)),
	"inh_weight_mask": None,

	"mapping": type('input_map', (object,), {'__getitem__': lambda s, x: np.array(x)})(),
	"firing_steps": -1,
	"input_pct_inhibitory": 0,

    "n_pools": 0,
	"output_range": [0, 1],

	"expected_value": lambda state: np.sum(state) % 2,
}

network = network_type(**parts, **network_config)

In [ ]:
# Alternative constructor approach, more readable when sharing to training loops and meta tools. Network template variables config and template parts have lower(lowest) priority than kwargs.
class network_template(network_type):
    config = network_config
    _template_parts = parts

network = network_template()

In [ ]:
network.reset()

for s in range(10):
    output_rates = network.tick(np.random.randint(2, size=2))

    print(output_rates)

## Games

A game is the framework for how agents can interact with some environment. Spikey contains two classes of games: RL and MetaRL. RL is for training spiking neural networks and MetaRL is for genetic algorithms.

All Spikey games are modelled after OpenAI gym environments, which can be converted to RL or MetaRL objects by the gym wrapper.

It is important to use Spikey's pre-built games or build off of a template to ensure compatability with training loops, callbacks and meta analysis tools.

```python
game_config = {
    'n_inputs': 2,
    'expected_value': lambda state: np.sum(state) % 2,
}
game = spikey.RL.Logic(**game_config)

state = game.reset()
for _ in range(10):
    action = network.tick(state)

    state, reward, done, info = game.step(action)
    print(state, '\t->', action)
    if done:
        break
```

In [ ]:
game_type = spikey.RL.Logic
game_type.NECESSARY_KEYS

In [ ]:
# Games may have a set of presets in game.PRESETS that set values for each key
game_type.PRESETS

In [ ]:
# Initialization using preset only
game = game_type(preset='XOR')

In [ ]:
# Are able to use kwargs similar to network, which take precedent over the preset
game_config = {
    'n_inputs': 2,
    'expected_value': lambda state: np.sum(state) % 2,
}
game = game_type(**game_config)

In [ ]:
# An alternative method, hardcoded game_template.config has lower(lowest) priority than presets and kwargs.
class game_template(game_type):
    config = game_config

game = game_template()
_ = game.seed(0)

In [ ]:
state = game.reset()
for _ in range(10):
    action = network.tick(state)

    state, reward, done, info = game.step(action)
    print(state, '\t->', action)
    if done:
        break

## Training Loops

Spiking neural networks are trained by having them interact with some environment, then giving them feedback based on their performance via some reinforcement or supervision signal.
In practice, the code that facilitates this process is called a training loop.
By the way Spikey is organized, many distinct experiments can be executed with the exact same training loops.
Therefore Spikey provides both pre-built and the ability for users to define their own training loops.

A TrainingLoop object exists that users may choose to base their implementations.
This handles some generally needed functionality of course, but more importantly it is the interface that all meta analysis tools expect.
```none
----------  -----------  ---------  -----
| Neuron |  | Synapse |  | Input |  | ...
----------  -----------  ---------  -----
       \         |         /
         \       |       /
--------   -------------
| Game |   |  Network  |
--------   -------------
   |            /
   |           /
-----------------
| Training Loop |
-----------------
```

In [ ]:
loop_type = spikey.core.GenericLoop
loop_type.NECESSARY_KEYS

In [ ]:
# Using pre-built, distributable training loop
experiment_params = {
    **network_config,
    **parts,
    **game_config,
    "n_episodes": 1,
    "len_episode": 10,
}
experiment = loop_type(network_type, game_type, experiment_params)

In [ ]:
# Creating a custom distributable training loop
class CustomLoop(spikey.core.TrainingLoop):
    NECESSARY_KEYS = spikey.core.TrainingLoop.extend_keys(
        {
            "n_episodes": "int Number of episodes to run,",
            "len_episode": "int Length of episode.",
        }
    )
    def __call__(self, **kwargs) -> (object, object, dict, dict):
        game = self.game_template(**self.params)
        network = self.network_template(game=game, **self.params)

        for e in range(self.params["n_episodes"]):
            network.reset()
            state = game.reset()

            for s in range(self.params["len_episode"]):
                action = network.tick(state)

                state, _, done, __ = game.step(action)

                if hasattr(network, 'reward') and callable(getattr(network, 'reward')):
                    reward = network.reward(state, action)

                if done:
                    break

        return network, game, {}, {}

not_experiment = CustomLoop(network_type, game_type, experiment_params)

In [ ]:
# Fully custom training loop
# NOTE: This method not recomended if you intend to use it in any meta analysis tools.
game = game_template()
network = network_template(game=game)

for e in range(experiment_params["n_episodes"]):
    network.reset()
    state = game.reset()

    for s in range(experiment_params["len_episode"]):
        action = network.tick(state)

        state, _, done, __ = game.step(action)

        if hasattr(network, 'reward') and callable(getattr(network, 'reward')):
            reward = network.reward(state, action)

        print(state, '\t->', action)

        if done:
            break

In [ ]:
# Results and info come from callback(below)
# Results is data that can be loaded to a table, info is things that require more analysis
experiment.reset()
network, game, results, info = experiment()

print(network)
print(game)
print(results)

## Callback

When experimenting, it is necessary to document initial conditions and track various signals.
In Spikey the Callback fills this need, which are capable of tracking preset as well as user defined signals.
Any data gathered by the Callback can be simply used, logged to file and read later by the respective tools.

```none
--------   -------------
| Game |   |  Network  |
--------   -------------    ------------
   |____________/___________| Callback | --> Logger --> Reader
   |           /            ------------
-----------------
| Training Loop |
-----------------
```

In [ ]:
# A standalone callback working on an arbitrary function, demonstrating how it is used by network and game
callback = spikey.core.ExperimentCallback()
callback.random = lambda *a: None  # Define binding

In [ ]:
## First add some signal trackers that trigger when binding is called in form
# callback.track(
#       binding_fn_name,
#       output location = 'result'/'info',
#       name for tracker,
#       target in form ['network/game/experiment_params', 'obj1', 'obj2', 'var_name'] or ['arg_#'],
#       method = 'list'/'scalar,       
# )

# Callback update a scalar variable in results dictionary whenever binding called
callback.track("random", "results", "demo_1", ['arg_0'], "scalar")

# Callback append to list in info dictionary whenever binding called
callback.track("random", "info", "demo_2", ['arg_0'], "list")

In [ ]:
callback.reset()

def random(arg1):
    callback.random(arg1)

In [ ]:
random(0)

print(f"demo_1: {callback.results['demo_1']}, demo_2: {callback.info['demo_2']}")

In [ ]:
callback.reset()
print(f"demo_1: {callback.results['demo_1']}, demo_2: {callback.info['demo_2']}")

for i in range(5):
    random(i)

print(f"demo_1: {callback.results['demo_1']}, demo_2: {callback.info['demo_2']}")

In [ ]:
# Further callback custom tracking
# RLCallback contains many preset trackers
callback = spikey.core.RLCallback()

# Object member variable
callback.track("network_init", "info", "weights_original", ["network", "synapses", "weights", "matrix"], "scalar")
callback.track("training_end", "results", "len_episode", ["experiment_params", "len_episode"], "scalar")

# Function argument
callback.track("network_reward", "info", "step_rewards", ["arg_2"], "list")

# Arbitrary function
callback.track("network_init", "info", "start_time", time, "scalar")

In [ ]:
experiment_params = {
    **network_config,
    **parts,
    **game_config,
    "n_episodes": 1,
    "len_episode": 10,
}
experiment = loop_type(network_type, game_type, experiment_params)

In [ ]:
# Log from tl and base
# This will save to a json file with members
# network: All network parts and serializable(json safe) parameters
# game: All game serializable(json safe) parameters.
# results: Any results that can be directly read into a table, typically from a callback
# info: Any other data relevant for analysis, typcially from a callback
filename = 'tutorial_log.json'

experiment.reset()
network, game, results, info = experiment(callback=callback)

callback.log(filename=filename)
# alternatively, spikey.logging.log(network, game, results, info, filename=filename)

In [ ]:
# Reader usage
reader = spikey.logging.Reader('.', [filename])

# For all items in network, game, result
reader.df

In [ ]:
# For specific items in any section
# [file1_results, file2_results, ...]
reader['weights_final']

## Moving Forward

Along with the core SNN framework and training platform, Spikey contains meta analysis tools(eg genetic algorithm), a small vizualization set and pre-configured experiments in examples/ and `spikey.experiments`.

Compared to deep neural networks, spiking neural networks are much closer to complex, dynamical systems than statistical tools. Interestingly they have the natural ability to reason about temporal information like series of events playing out over time. Internally they are able to encode information not only by the magnitudes of neuron fires but also into firing rates and temporal patterns! These attributes make SNNs ideal models for reinforcement learning, control tasks and more!

Please share any difficulties or suggestions with Spikey or your spiking network related research in our issue tab so that we can help.

Further introduction to SNNs,

* [Paugam-Moisy H., Bohte S.(2012) Computing with Spiking Neuron Networks. In: Rozenberg G., Bäck T., Kok J.N. (eds) Handbook of Natural Computing. Springer, Berlin, Heidelberg. https://doi.org/10.1007/978-3-540-92910-9_10](https://homepages.cwi.nl/~sbohte/publication/paugam_moisy_bohte_SNNChapter.pdf)

* [A. Grüning, S. Bohté(2014) Spiking Neural Networks: Principles and Challenges. ESAN.](https://www.elen.ucl.ac.be/Proceedings/esann/esannpdf/es2014-13.pdf)

* [What is a Spiking Neural Network? Spikey.](https://github.com/SpikeyCNS/spikey#spiking-neural-networks)

Some SNN applications,

* [Michael A. Farries and Adrienne L. Fairhall(2007)
Reinforcement Learning With Modulated Spike Timing–Dependent Synaptic Plasticity.
Journal of Neurophysiology 2007 98:6, 3648-3665.](https://journals.physiology.org/doi/pdf/10.1152/jn.00364.2007)

* [Florian R(2007) Reinforcement Learning Through Modulation of
Spike-Timing-Dependent Synaptic Plasticity. Neural Computation 19(6).
https://doi.org/10.1162/neco.2007.19.6.1468](https://www.florian.io/papers/2007_Florian_Modulated_STDP.pdf)

* [Stephen Chung and Robert Kozma(2020) Reinforcement Learning with Feedback-modulated TD-STDP. arXiv 2008.13044.](https://arxiv.org/abs/2008.13044)